# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     | Michelle Trigo  |
| **Fecha**      | 15/09/25  |
| **Expediente** | 745567  |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_excel("Motor Trend Car Road Tests.xlsx")
matriz = df.to_numpy()

In [4]:
df = df.drop(columns=["model"])
X = df.drop(columns=["mpg"])
y = df["mpg"]
X = X.astype(float)
y= y.astype(float)

In [5]:
n = len(y)
mse_scores = []

In [6]:
for i in range(n):
    # Separar train y test usando drop
    X_train = X.drop(i).to_numpy()
    y_train = y.drop(i).to_numpy()
    X_test = X.iloc[i].to_numpy().reshape(1, -1)
    y_test = y.iloc[i]
    scaler= StandardScaler().fit(X_train)
    X_scaled=scaler.transform(X_train)

    
    # Regresión
    model = LinearRegression()
    model.fit(X_scaled, y_train)
    
    # Predicción
    X_scaled2=scaler.transform(X_test)
    y_pred = model.predict(X_scaled2)
    
    # MSE 
    mse = mean_squared_error([y_test], y_pred)
    mse_scores.append(mse)

print("Número de iteraciones:", n)
print("MSE promedio:", np.mean(mse_scores))
print("Desviación estándar del MSE:", np.std(mse_scores))

Número de iteraciones: 32
MSE promedio: 12.181558006901941
Desviación estándar del MSE: 17.06739987188858


In [7]:
mse_scores

[5.258855796629049,
 2.453996713086807,
 20.550493416882723,
 0.0443291071396446,
 1.5811555159760176,
 10.11862548887538,
 0.0163740415484736,
 8.081250710389478,
 39.47018347964978,
 0.7706327722901085,
 4.955694234215832,
 10.224730198354477,
 4.430129331402683,
 0.4878127712478593,
 6.8238445758093995,
 0.6027506975546868,
 36.81017860995889,
 31.759855581210395,
 1.063227541805811,
 32.7132861870273,
 14.307173682712742,
 3.405354649998888,
 9.408007649079861,
 0.0001035003980367482,
 9.96243953454772,
 1.3412208919431232,
 0.16480064082223855,
 23.60020650224925,
 83.11457643385205,
 0.00010271237184860317,
 8.785533367966229,
 17.502929883865335]

**Interpreta**

El modelo lineal hace un trabajo razonable en promedio, con un error medio (MSE) de 12.18, pero los errores varían mucho entre autos. Algunos se predicen casi perfectamente, mientras que otros, con características extremas, tienen errores muy grandes. Esto significa que el modelo funciona bien en general, pero no es consistente para todos los autos y podría mejorar usando transformaciones de datos o modelos más complejos.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Interpreta.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3